**Projeto 2:  Stock Portfolio Forecasting and Optimization on S&P500 Using Machine Learning and Search Methods**

**Unidade Curicular:** Laboratório de Inteligência Artificial e Ciência de Dados

**Grupo 7:**

up202206252 Inês Neves

up202205826 Beatriz Fonseca

up202208293 Maria Beatriz Moreira

**Introdução**

O projeto realizado tem como objetivo prever os preços de ações de várias empresas do S&P500.

Para isso utilizamos algoritmos de aprendizagem clássicos (**AdaBoost, XGBoost e LGBM**) e de *deep learning* (**LSTM**).

Pretende-se, para além disso, selecionar os melhores *stocks* para o investimento diário, de forma a maximizar os retornos financeiros, razão pela qual recorremos ao **Monte Carlo** como método de otimização.


**Etapas realizadas**


-> Extração dos dados relativos a todas as empresas presentes no S&P 500;



-> **COMPLETAR**



-> Realização de *Feature Engineering*;

-> Realização de uma análise explorativa dos dados;

-> Seleção e implementação dos algoritmos a utilizar: **XGBoost**, **AdaBoost**, **LGBM**, **LSTM**.

-> Implementação do **Monte Carlo** como método de otimização;

-> Análise dos resultados.

**Bibliotecas Necessárias**

In [8]:
import os
import pandas as pd
import pandas_ta as ta
import yfinance as yf
from pathlib import Path
from loguru import logger
from fancyimpute import IterativeImputer
from sklearn.preprocessing import LabelEncoder

**Parte 1: Extração de dados**

Inicialmente, definimos a função *get_sp500_tickers* com o objetivo de obter a lista dos tickers das empresas que compõem o índice S&P 500. Necessitamos destes tickers para posteriormente os utilizarmos nos dados provenientes no yahoo finance.

In [ ]:
def get_sp500_tickers():
    #URL da página da Wikipédia para a lista de empresas S&P 500
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

    #Utilizar o pandas para ler a tabela que contém os tickers
    tables = pd.read_html(url)
    df = tables[0]

    #Extrair a coluna de símbolos de ticker
    tickers = df['Symbol'].tolist()

    #Remove as linhas com tickers em falta
    tickers = [ticker for ticker in tickers if ticker != '']

    return tickers

sp500_tickers = get_sp500_tickers()
print(sp500_tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMTM', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', '

In [ ]:
#Configurações
CONFIG = {
    "indices": [
        'MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM',
        'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMTM', 'AEE',
        'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL',
        'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY',
        'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA',
        'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR',
        'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR',
        'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C',
        'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT',
        'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DVA',
        'DAY', 'DECK', 'DE', 'DELL', 'DAL', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV',
        'DOW', 'DHI', 'DTE', 'DUK', 'DD', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'ELV', 'EMR', 'ENPH',
        'ETR', 'EOG', 'EPAM', 'EQT', 'EFX', 'EQIX', 'EQR', 'ERIE', 'ESS', 'EL', 'EG', 'EVRG', 'ES', 'EXC', 'EXPE',
        'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FICO', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FSLR', 'FE', 'FI', 'FMC',
        'F', 'FTNT', 'FTV', 'FOXA', 'FOX', 'BEN', 'FCX', 'GRMN', 'IT', 'GE', 'GEHC', 'GEV', 'GEN', 'GNRC', 'GD',
        'GIS', 'GM', 'GPC', 'GILD', 'GPN', 'GL', 'GDDY', 'GS', 'HAL', 'HIG', 'HAS', 'HCA', 'DOC', 'HSIC', 'HSY',
        'HES', 'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUBB', 'HUM', 'HBAN', 'HII', 'IBM',
        'IEX', 'IDXX', 'ITW', 'INCY', 'IR', 'PODD', 'INTC', 'ICE', 'IFF', 'IP', 'IPG', 'INTU', 'ISRG', 'IVZ', 'INVH',
        'IQV', 'IRM', 'JBHT', 'JBL', 'JKHY', 'J', 'JNJ', 'JCI', 'JPM', 'JNPR', 'K', 'KVUE', 'KDP', 'KEY', 'KEYS',
        'KMB', 'KIM', 'KMI', 'KKR', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LDOS', 'LEN', 'LLY',
        'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LULU', 'LYB', 'MTB', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS',
        'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'META', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA',
        'MRNA', 'MHK', 'MOH', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP',
        'NFLX', 'NEM', 'NWSA', 'NWS', 'NEE', 'NKE', 'NI', 'NDSN', 'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE',
        'NVDA', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL', 'OMC', 'ON', 'OKE', 'ORCL', 'OTIS', 'PCAR', 'PKG', 'PLTR',
        'PANW', 'PARA', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PEP', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PNC',
        'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PTC', 'PSA', 'PHM', 'QRVO', 'PWR', 'QCOM',
        'DGX', 'RL', 'RJF', 'RTX', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RVTY', 'ROK', 'ROL', 'ROP', 'ROST',
        'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SJM', 'SW', 'SNA', 'SOLV',
        'SO', 'LUV', 'SWK', 'SBUX', 'STT', 'STLD', 'STE', 'SYK', 'SMCI', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW',
        'TTWO', 'TPR', 'TRGP', 'TGT', 'TEL', 'TDY', 'TFX', 'TER', 'TSLA', 'TXN', 'TPL', 'TXT', 'TMO', 'TJX', 'TSCO',
        'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TYL', 'TSN', 'USB', 'UBER', 'UDR', 'ULTA', 'UNP', 'UAL', 'UPS', 'URI',
        'UNH', 'UHS', 'VLO', 'VTR', 'VLTO', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VTRS', 'VICI', 'V', 'VST', 'VMC', 'WRB',
        'GWW', 'WAB', 'WBA', 'WMT', 'DIS', 'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WHR', 'XEL', 'XYL', 'YUM',
        'ZTS'
    ],
    "start_date": "2009-08-31",
    "end_date": "2024-12-31",
    "data_dir": Path("updated_data")
}

#Assegurar que o diretório de dados existe
CONFIG['data_dir'].mkdir(parents=True, exist_ok=True)

def download_stock_data(symbols, start_date, end_date, data_dir):
    for symbol in symbols:
        try:
            logger.info(f"Downloading data for {symbol}...")
            data = yf.download(symbol, start=start_date, end=end_date)

            #Guardar os dados num ficheiro CSV
            file_path = data_dir / f"{symbol}.csv"
            data.to_csv(file_path)
            logger.info(f"Data for {symbol} saved to {file_path}")
        except Exception as e:
            logger.error(f"Failed to download data for {symbol}: {e}")


download_stock_data(CONFIG["indices"], CONFIG["start_date"], CONFIG["end_date"], CONFIG["data_dir"])


2024-12-10 16:57:55.262 | INFO     | __main__:download_stock_data:51 - Downloading data for MMM...
[*********************100%***********************]  1 of 1 completed
2024-12-10 16:57:56.124 | INFO     | __main__:download_stock_data:57 - Data for MMM saved to updated_data\MMM.csv
2024-12-10 16:57:56.128 | INFO     | __main__:download_stock_data:51 - Downloading data for AOS...
[*********************100%***********************]  1 of 1 completed
2024-12-10 16:57:56.451 | INFO     | __main__:download_stock_data:57 - Data for AOS saved to updated_data\AOS.csv
2024-12-10 16:57:56.452 | INFO     | __main__:download_stock_data:51 - Downloading data for ABT...
[*********************100%***********************]  1 of 1 completed
2024-12-10 16:57:56.801 | INFO     | __main__:download_stock_data:57 - Data for ABT saved to updated_data\ABT.csv
2024-12-10 16:57:56.801 | INFO     | __main__:download_stock_data:51 - Downloading data for ABBV...
[*********************100%***********************]  1 

Formatação do conteúdo dos ficheiros por empresa

In [ ]:
# Carregar o dataset
abt = pd.read_csv(r"C:\Users\beatr\Music\updated_data\ABT.csv")

abt.head(5)

,Price,Adj Close,Close,High,Low,Open,Volume
0,Ticker,ABT,ABT,ABT,ABT,ABT,ABT
1,Date,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-08-31 00:00:00+00:00,15.458928108215332,21.701353073120117,22.200345993041992,21.600595474243164,21.960445404052734,20529596
3,2009-09-01 00:00:00+00:00,15.421334266662598,21.648576736450195,21.94605255126953,21.514232635498047,21.629383087158203,15665069
4,2009-09-02 00:00:00+00:00,15.380317687988281,21.590999603271484,21.778121948242188,21.456655502319336,21.638980865478516,14121509


In [ ]:
input_folder = r"C:\Users\beatr\Music\updated_data"
output_folder = r"C:\Users\beatr\Music\new_updated_data"

#Iterar sobre todos os ficheiros na pasta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_folder, filename)

        df = pd.read_csv(file_path)

        #Adicionar a coluna "Ticker" com o nome do ficheiro
        ticker = os.path.splitext(filename)[0]
        df["Ticker"] = ticker

        #Renomear e reorganizar as colunas
        df.columns = ["Date", "Adj Close", "Close", "High", "Low", "Open", "Volume", "Ticker"]
        df = df[["Date", "Adj Close", "Close", "High", "Low", "Open", "Volume", "Ticker"]]

        #Remover as duas primeiras linhas
        df = df.iloc[2:]

        #Guardar o ficheiro processado na pasta de saída
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)

        print(f"Processado: {filename} -> {output_path}")

print("Processamento concluído!")

Processado: A.csv -> C:\Users\beatr\Music\new_updated_data\A.csv
Processado: AAPL.csv -> C:\Users\beatr\Music\new_updated_data\AAPL.csv
Processado: ABBV.csv -> C:\Users\beatr\Music\new_updated_data\ABBV.csv
Processado: ABNB.csv -> C:\Users\beatr\Music\new_updated_data\ABNB.csv
Processado: ABT.csv -> C:\Users\beatr\Music\new_updated_data\ABT.csv
Processado: ACGL.csv -> C:\Users\beatr\Music\new_updated_data\ACGL.csv
Processado: ACN.csv -> C:\Users\beatr\Music\new_updated_data\ACN.csv
Processado: ADBE.csv -> C:\Users\beatr\Music\new_updated_data\ADBE.csv
Processado: ADI.csv -> C:\Users\beatr\Music\new_updated_data\ADI.csv
Processado: ADM.csv -> C:\Users\beatr\Music\new_updated_data\ADM.csv
Processado: ADP.csv -> C:\Users\beatr\Music\new_updated_data\ADP.csv
Processado: ADSK.csv -> C:\Users\beatr\Music\new_updated_data\ADSK.csv
Processado: AEE.csv -> C:\Users\beatr\Music\new_updated_data\AEE.csv
Processado: AEP.csv -> C:\Users\beatr\Music\new_updated_data\AEP.csv
Processado: AES.csv -> C:\

Junção dos dados de todas as empresas num só dataset

In [ ]:
input_folder =  r"C:\Users\beatr\Music\new_updated_data"
output_file =  r"C:\Users\beatr\Music\\NEW_NEW_dataset.csv"

# Lista para armazenar os DataFrames
dataframes = []

# Iterar sobre todos os ficheiros CSV na pasta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_folder, filename)

        df = pd.read_csv(file_path)
        dataframes.append(df)

        print(f"Arquivo processado: {filename}")

# Concatenar todos os DataFrames
df_consolidado = pd.concat(dataframes, ignore_index=True)

# Guardar o DataFrame consolidado num único ficheiro CSV
df_consolidado.to_csv(output_file, index=False)

print(f"Arquivo consolidado salvo como: {output_file}")

Arquivo processado: A.csv
Arquivo processado: AAPL.csv
Arquivo processado: ABBV.csv
Arquivo processado: ABNB.csv
Arquivo processado: ABT.csv
Arquivo processado: ACGL.csv
Arquivo processado: ACN.csv
Arquivo processado: ADBE.csv
Arquivo processado: ADI.csv
Arquivo processado: ADM.csv
Arquivo processado: ADP.csv
Arquivo processado: ADSK.csv
Arquivo processado: AEE.csv
Arquivo processado: AEP.csv
Arquivo processado: AES.csv
Arquivo processado: AFL.csv
Arquivo processado: AIG.csv
Arquivo processado: AIZ.csv
Arquivo processado: AJG.csv
Arquivo processado: AKAM.csv
Arquivo processado: ALB.csv
Arquivo processado: ALGN.csv
Arquivo processado: ALL.csv
Arquivo processado: ALLE.csv
Arquivo processado: AMAT.csv
Arquivo processado: AMCR.csv
Arquivo processado: AMD.csv
Arquivo processado: AME.csv
Arquivo processado: AMGN.csv
Arquivo processado: AMP.csv
Arquivo processado: AMT.csv
Arquivo processado: AMTM.csv
Arquivo processado: AMZN.csv
Arquivo processado: ANET.csv
Arquivo processado: ANSS.csv
Arquiv

In [ ]:
# Carregar o dataset
df = pd.read_csv("NEW_NEW_dataset.csv")

# Indicadores Técnicos:

- ***Médias Móveis Simples** (SMA_20, SMA_50)*: Calculam a média do preço ajustado dos últimos 20 e 50 períodos, respectivamente. São usados para identificar tendências e suavizar flutuações de curto prazo.

- **Média Móvel Exponencial** (EMA_20): Dá mais peso aos preços recentes em comparação à SMA, sendo útil para identificar tendências com maior sensibilidade a alterações recentes.

- **RSI** (Relative Strength Index): Mede a força relativa de um ativo ao longo de 14 períodos, ajudando a identificar condições de sobrecompra ou sobrevenda.

- **Bandas de Bollinger**: Usam uma média móvel de 20 períodos (BB_Middle) e dois desvios padrão para calcular os limites superior e inferior (BB_Upper, BB_Lower), que ajudam a avaliar a volatilidade e a identificar possíveis reversões.

- **Momentum** (Momentum_10): Mede a variação do preço ajustado em 10 períodos, avaliando a força e a direção do movimento dos preços.

- **ATR** (Average True Range): Calcula a média da amplitude diária de preços ao longo de 14 períodos, representando a volatilidade média.

- **Volatilidade** (Desvio Padrão de 20 períodos): Mede a dispersão do preço ajustado em 20 períodos, indicando o grau de incerteza ou risco.

- **Retornos diários** (Daily_Return): Calculam a variação percentual diária do preço ajustado, usados para a análise de desempenho e  da volatilidade.

- **Retornos futuros** (Future_Return): Antecipam a variação percentual futura do preço ajustado, servindo como variável alvo para modelos de previsão.

- **Volatilidade Móvel** (Rolling_Volatility): Mede o desvio padrão dos retornos diários em janelas móveis de 10 dias, fornecendo uma visão mais detalhada da volatilidade ao longo do tempo.

In [ ]:
# Garantir que a coluna "Date" está em formato datetime
df['Date'] = pd.to_datetime(df['Date'])

# Ordenar os dados por Data e Ticker
df = df.sort_values(by=["Date", "Ticker"])

# Adicionar indicadores técnicos ao dataset
# 1. Médias Móveis (20, 50 períodos)
df['SMA_20'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: x.rolling(window=20).mean())
df['SMA_50'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: x.rolling(window=50).mean())
df['EMA_20'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: x.ewm(span=20, adjust=False).mean())

# Preencher valores NaN gerados pelas médias móveis
df['SMA_20'].fillna(method='ffill', inplace=True)
df['SMA_50'].fillna(method='ffill', inplace=True)
df['EMA_20'].fillna(method='ffill', inplace=True)

# 2. RSI (Relative Strength Index) - Padrão: 14 períodos
df['RSI'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: ta.momentum.rsi(x, window=14))

# 3. Bandas de Bollinger (20 períodos, 2 desvios padrão)
df['BB_Middle'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: x.rolling(window=20).mean())
df['BB_Upper'] = df['BB_Middle'] + 2 * df.groupby('Ticker')['Adj Close'].transform(lambda x: x.rolling(window=20).std())
df['BB_Lower'] = df['BB_Middle'] - 2 * df.groupby('Ticker')['Adj Close'].transform(lambda x: x.rolling(window=20).std())

# Preencher valores NaN das Bandas de Bollinger
df['BB_Middle'].fillna(method='ffill', inplace=True)
df['BB_Upper'].fillna(method='ffill', inplace=True)
df['BB_Lower'].fillna(method='ffill', inplace=True)

# 4. Momentum (10 períodos)
df['Momentum_10'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: x.diff(periods=10))

# 5. ATR (Average True Range, 14 períodos)
df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)

# 6. Volatilidade (Desvio Padrão de 20 períodos)
df['Volatility'] = df.groupby('Ticker')['Adj Close'].transform(lambda x: x.rolling(window=20).std())

# Preencher valores NaN da volatilidade
df['Volatility'].fillna(method='ffill', inplace=True)

# Calculate daily returns
df['Daily_Return'] = df.groupby('Ticker')['Adj Close'].pct_change()

# Calculate future returns as the target
df['Future_Return'] = df.groupby('Ticker')['Adj Close'].pct_change().shift(-1)

# Rolling volatility (e.g., 10-day standard deviation)
df['Rolling_Volatility'] = df.groupby('Ticker')['Daily_Return'].rolling(window=10).std().reset_index(0, drop=True)


df.to_csv("updated_dataset.csv", index=False)

In [ ]:
# Carregar o dataset
df = pd.read_csv("updated_dataset.csv")

df = df[~df['Date'].str.startswith('2009')]

df.to_csv("final_dataset.csv", index=False)

Os dados de 2009 foram incluídos para garantir o cálculo correto dos indicadores técnicos, que requerem períodos anteriores para médias móveis, RSI, entre  outros. \
Após os cálculos, estes dados foram removidos.

**Pré-processamento dos dados**

In [18]:
#Carregar o dataset
df = pd.read_csv("final_dataset.csv")
df

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,SMA_20,SMA_50,...,RSI,BB_Middle,BB_Upper,BB_Lower,Momentum_10,ATR,Volatility,Daily_Return,Future_Return,Rolling_Volatility
0,2010-01-04 00:00:00+00:00,20.053024,22.389128,22.625179,22.267525,22.453505,3815561,A,19.262117,18.290923,...,73.322938,19.262117,19.993784,18.530449,1.191645,22.892235,0.365834,0.007402,0.015566,0.009005
1,2010-01-04 00:00:00+00:00,6.447413,7.643214,7.660714,7.585000,7.622500,493729600,AAPL,6.027340,6.015485,...,67.311129,6.027340,6.510970,5.543711,0.667307,22.314513,0.241815,0.015566,0.008705,0.012471
2,2010-01-04 00:00:00+00:00,18.763710,26.129908,26.177889,25.870815,26.000362,10829095,ABT,18.583864,18.309058,...,58.179645,18.583864,18.831443,18.336284,0.303192,22.044525,0.123790,0.008705,0.005590,0.006702
3,2010-01-04 00:00:00+00:00,7.601905,7.994444,8.022222,7.972222,7.978889,4813200,ACGL,7.490492,7.385396,...,60.654008,7.490492,7.701626,7.279357,0.159539,21.766893,0.105567,0.005590,0.013735,0.007004
4,2010-01-04 00:00:00+00:00,31.941805,42.070000,42.200001,41.500000,41.520000,3650100,ACN,31.756157,30.628892,...,59.125958,31.756157,32.670683,30.841632,0.311298,22.655369,0.457263,0.013735,0.008429,0.009659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755809,2024-12-10 00:00:00+00:00,69.278297,69.278297,69.430000,68.220001,69.110001,708077,XEL,70.438915,66.887566,...,50.035062,70.438915,73.014700,67.863130,-2.321701,134.768733,1.287893,0.002435,0.001949,0.009790
1755810,2024-12-10 00:00:00+00:00,113.120003,113.120003,113.879997,111.750000,113.584999,8462100,XOM,118.021456,118.950867,...,33.327459,118.021456,123.733793,112.309119,-6.849998,128.328231,2.856168,0.001949,0.001237,0.010303
1755811,2024-12-10 00:00:00+00:00,129.509995,129.509995,129.940002,126.800003,129.910004,508934,XYL,125.387312,128.336431,...,59.331245,125.387312,130.615718,120.158907,0.929993,120.363357,2.614203,0.001237,-0.006112,0.014281
1755812,2024-12-10 00:00:00+00:00,138.229996,138.229996,138.789993,136.479996,138.610001,343854,YUM,136.107500,134.862716,...,57.837375,136.107500,141.112670,131.102329,1.840881,112.428831,2.502585,-0.006112,-0.000568,0.006979


Verificar valores nulos

In [19]:
#Check if the dataset has any missing values
print(df.isnull().sum())

Date                     0
Adj Close                0
Close                    0
High                     0
Low                      0
Open                     0
Volume                   0
Ticker                   0
SMA_20                   0
SMA_50                   0
EMA_20                   0
RSI                   1024
BB_Middle                0
BB_Upper                 0
BB_Lower                 0
Momentum_10            730
ATR                      0
Volatility               0
Daily_Return            73
Future_Return           74
Rolling_Volatility     730
dtype: int64


In [20]:
#Agrupar por Ticker e contar valores nulos para as colunas de interesse
null_counts_by_ticker = df.groupby('Ticker')[['RSI', 'Momentum_10', 'Daily_Return', 'Future_Return', 'Rolling_Volatility']].apply(lambda x: x.isnull().sum())

#Filtrar Tickers que não têm nulos, em todas as colunas de interesse
filtered_null_counts = null_counts_by_ticker[(null_counts_by_ticker['RSI'] > 0) |
                                             (null_counts_by_ticker['Momentum_10'] > 0) |
                                             (null_counts_by_ticker['Daily_Return'] > 0) |
                                             (null_counts_by_ticker['Future_Return'] > 0) |
                                             (null_counts_by_ticker['Rolling_Volatility'] > 0)]

#Exibir os resultados
filtered_null_counts.head(20)

,RSI,Momentum_10,Daily_Return,Future_Return,Rolling_Volatility
Ticker,,,,,
AAPL,0,0,0,1,0
ABBV,14,10,1,1,10
ABNB,14,10,1,0,10
ALL,0,0,0,1,0
ALLE,14,10,1,0,10
AMAT,0,0,0,1,0
AMCR,16,10,1,0,10
AMT,0,0,0,1,0
AMTM,14,10,1,0,10


Este estudo fornece uma abordagem valiosa para o tratamento de valores em falta em dados de séries temporais, particularmente em contextos financeiros e económicos, demonstrando a sua potencial aplicação em vários domínios que lidam com dados de séries temporais, contendo valores em falta.

O algoritmo de imputação teve um bom desempenho na previsão dos preços das acções para o preço de abertura após um feriado, superando o método de referência.

https://www.proquest.com/openview/5c952be644ccd065c3f5bcd4e4381b49/1?pq-origsite=gscholar&cbl=18750

# REVER PODE SER MELHOR ELIMINAR ESTA PARTE
A imputação múltipla é uma técnica estatística eficaz para tratar dados ausentes, especialmente em séries temporais, onde métodos simples como substituição por média podem distorcer a estrutura dos dados. Este método gera várias estimativas para os valores ausentes, considerando a incerteza associada, e combina os resultados para maior precisão.

O método MICE (Multiple Imputation by Chained Equations), amplamente utilizado, imputa iterativamente os dados ausentes com base em outras variáveis, preservando relações complexas e variabilidade dos dados. É particularmente útil em séries temporais financeiras, onde mantém correlações temporais e melhora a qualidade de previsões de ações ou índices financeiros.

Ferramentas como a biblioteca fancyimpute em Python permitem aplicar o MICE de forma robusta, assegurando análises mais precisas e confiáveis. Estudos como os de Rubin (1987) e Oh (2015) destacam a relevância desta abordagem para melhorar a integridade e a qualidade das análises preditivas.

*Referência:* Oh, S. (2015). Multiple Imputation on Missing Values in Time Series Data.


Utilização do MICE para imputar valores nas entradas nulas

In [21]:
#Selecionar apenas as colunas com valores nulos
cols_to_impute = ['RSI', 'Momentum_10', 'Daily_Return', 'Future_Return', 'Rolling_Volatility']

#Definir o imputador múltiplo (IterativeImputer usa MICE)
imputador = IterativeImputer(max_iter=10, random_state=0)

#Aplicar imputação múltipla
df[cols_to_impute] = imputador.fit_transform(df[cols_to_impute])

#Verificar se os valores nulos foram preenchidos
print(df.isnull().sum())

Date                  0
Adj Close             0
Close                 0
High                  0
Low                   0
Open                  0
Volume                0
Ticker                0
SMA_20                0
SMA_50                0
EMA_20                0
RSI                   0
BB_Middle             0
BB_Upper              0
BB_Lower              0
Momentum_10           0
ATR                   0
Volatility            0
Daily_Return          0
Future_Return         0
Rolling_Volatility    0
dtype: int64


In [22]:
#Verficar informações do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1755814 entries, 0 to 1755813
Data columns (total 21 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Date                object 
 1   Adj Close           float64
 2   Close               float64
 3   High                float64
 4   Low                 float64
 5   Open                float64
 6   Volume              int64  
 7   Ticker              object 
 8   SMA_20              float64
 9   SMA_50              float64
 10  EMA_20              float64
 11  RSI                 float64
 12  BB_Middle           float64
 13  BB_Upper            float64
 14  BB_Lower            float64
 15  Momentum_10         float64
 16  ATR                 float64
 17  Volatility          float64
 18  Daily_Return        float64
 19  Future_Return       float64
 20  Rolling_Volatility  float64
dtypes: float64(18), int64(1), object(2)
memory usage: 281.3+ MB


Normalizar os dados

In [23]:
# Converter a coluna 'Date' para o tipo datetime, caso ainda não esteja
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Extrair componentes temporais (ano, mês, dia, dia da semana)
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.weekday + 2  # Segunda-feira é 0, Domingo é 6

In [24]:
# Visualizar as primeiras linhas
df[['Date','Year', 'Month', 'Day', 'Weekday']].head(20)

,Date,Year,Month,Day,Weekday
0,2010-01-04 00:00:00+00:00,2010,1,4,2
1,2010-01-04 00:00:00+00:00,2010,1,4,2
2,2010-01-04 00:00:00+00:00,2010,1,4,2
3,2010-01-04 00:00:00+00:00,2010,1,4,2
4,2010-01-04 00:00:00+00:00,2010,1,4,2
5,2010-01-04 00:00:00+00:00,2010,1,4,2
6,2010-01-04 00:00:00+00:00,2010,1,4,2
7,2010-01-04 00:00:00+00:00,2010,1,4,2
8,2010-01-04 00:00:00+00:00,2010,1,4,2
9,2010-01-04 00:00:00+00:00,2010,1,4,2


Weekdays:

2- segunda;
3- terça;
4- quarta;
5- quinta;
6- sexta;
7- sábado;
8- domingo;

Coluna ticket

In [25]:
# Criar o LabelEncoder
label_encoder = LabelEncoder()

# Aplicar o Label Encoding à coluna 'Ticker'
df['Ticker'] = label_encoder.fit_transform(df['Ticker'])

df

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,SMA_20,SMA_50,...,Momentum_10,ATR,Volatility,Daily_Return,Future_Return,Rolling_Volatility,Year,Month,Day,Weekday
0,2010-01-04 00:00:00+00:00,20.053024,22.389128,22.625179,22.267525,22.453505,3815561,0,19.262117,18.290923,...,1.191645,22.892235,0.365834,0.007402,0.015566,0.009005,2010,1,4,2
1,2010-01-04 00:00:00+00:00,6.447413,7.643214,7.660714,7.585000,7.622500,493729600,1,6.027340,6.015485,...,0.667307,22.314513,0.241815,0.015566,0.008705,0.012471,2010,1,4,2
2,2010-01-04 00:00:00+00:00,18.763710,26.129908,26.177889,25.870815,26.000362,10829095,4,18.583864,18.309058,...,0.303192,22.044525,0.123790,0.008705,0.005590,0.006702,2010,1,4,2
3,2010-01-04 00:00:00+00:00,7.601905,7.994444,8.022222,7.972222,7.978889,4813200,5,7.490492,7.385396,...,0.159539,21.766893,0.105567,0.005590,0.013735,0.007004,2010,1,4,2
4,2010-01-04 00:00:00+00:00,31.941805,42.070000,42.200001,41.500000,41.520000,3650100,6,31.756157,30.628892,...,0.311298,22.655369,0.457263,0.013735,0.008429,0.009659,2010,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755809,2024-12-10 00:00:00+00:00,69.278297,69.278297,69.430000,68.220001,69.110001,708077,489,70.438915,66.887566,...,-2.321701,134.768733,1.287893,0.002435,0.001949,0.009790,2024,12,10,3
1755810,2024-12-10 00:00:00+00:00,113.120003,113.120003,113.879997,111.750000,113.584999,8462100,490,118.021456,118.950867,...,-6.849998,128.328231,2.856168,0.001949,0.001237,0.010303,2024,12,10,3
1755811,2024-12-10 00:00:00+00:00,129.509995,129.509995,129.940002,126.800003,129.910004,508934,491,125.387312,128.336431,...,0.929993,120.363357,2.614203,0.001237,-0.006112,0.014281,2024,12,10,3
1755812,2024-12-10 00:00:00+00:00,138.229996,138.229996,138.789993,136.479996,138.610001,343854,492,136.107500,134.862716,...,1.840881,112.428831,2.502585,-0.006112,-0.000568,0.006979,2024,12,10,3


In [26]:
print("Classes no LabelEncoder:", label_encoder.classes_)


Classes no LabelEncoder: ['A' 'AAPL' 'ABBV' 'ABNB' 'ABT' 'ACGL' 'ACN' 'ADBE' 'ADI' 'ADM' 'ADP'
 'ADSK' 'AEE' 'AEP' 'AES' 'AFL' 'AIG' 'AIZ' 'AJG' 'AKAM' 'ALB' 'ALGN'
 'ALL' 'ALLE' 'AMAT' 'AMCR' 'AMD' 'AME' 'AMGN' 'AMP' 'AMT' 'AMTM' 'AMZN'
 'ANET' 'ANSS' 'AON' 'AOS' 'APA' 'APD' 'APH' 'APTV' 'ARE' 'ATO' 'AVB'
 'AVGO' 'AVY' 'AWK' 'AXON' 'AXP' 'AZO' 'BA' 'BAC' 'BALL' 'BAX' 'BBY' 'BDX'
 'BEN' 'BG' 'BIIB' 'BK' 'BKNG' 'BKR' 'BLDR' 'BLK' 'BMY' 'BR' 'BRO' 'BSX'
 'BWA' 'BX' 'BXP' 'C' 'CAG' 'CAH' 'CARR' 'CAT' 'CB' 'CBOE' 'CBRE' 'CCI'
 'CCL' 'CDNS' 'CDW' 'CE' 'CEG' 'CF' 'CFG' 'CHD' 'CHRW' 'CHTR' 'CI' 'CINF'
 'CL' 'CLX' 'CMCSA' 'CME' 'CMG' 'CMI' 'CMS' 'CNC' 'CNP' 'COF' 'COO' 'COP'
 'COR' 'COST' 'CPAY' 'CPB' 'CPRT' 'CPT' 'CRL' 'CRM' 'CRWD' 'CSCO' 'CSGP'
 'CSX' 'CTAS' 'CTLT' 'CTRA' 'CTSH' 'CTVA' 'CVS' 'CVX' 'CZR' 'D' 'DAL'
 'DAY' 'DD' 'DE' 'DECK' 'DELL' 'DFS' 'DG' 'DGX' 'DHI' 'DHR' 'DIS' 'DLR'
 'DLTR' 'DOC' 'DOV' 'DOW' 'DPZ' 'DRI' 'DTE' 'DUK' 'DVA' 'DVN' 'DXCM' 'EA'
 'EBAY' 'ECL' 'ED' 'EFX' 'EG' 'EIX'

In [28]:
# Armazenar os nomes originais dos tickers correspondentes aos números
ticker_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("\nMapping dos Tickers:")
print(ticker_mapping)


Mapping dos Tickers:
{'A': 0, 'AAPL': 1, 'ABBV': 2, 'ABNB': 3, 'ABT': 4, 'ACGL': 5, 'ACN': 6, 'ADBE': 7, 'ADI': 8, 'ADM': 9, 'ADP': 10, 'ADSK': 11, 'AEE': 12, 'AEP': 13, 'AES': 14, 'AFL': 15, 'AIG': 16, 'AIZ': 17, 'AJG': 18, 'AKAM': 19, 'ALB': 20, 'ALGN': 21, 'ALL': 22, 'ALLE': 23, 'AMAT': 24, 'AMCR': 25, 'AMD': 26, 'AME': 27, 'AMGN': 28, 'AMP': 29, 'AMT': 30, 'AMTM': 31, 'AMZN': 32, 'ANET': 33, 'ANSS': 34, 'AON': 35, 'AOS': 36, 'APA': 37, 'APD': 38, 'APH': 39, 'APTV': 40, 'ARE': 41, 'ATO': 42, 'AVB': 43, 'AVGO': 44, 'AVY': 45, 'AWK': 46, 'AXON': 47, 'AXP': 48, 'AZO': 49, 'BA': 50, 'BAC': 51, 'BALL': 52, 'BAX': 53, 'BBY': 54, 'BDX': 55, 'BEN': 56, 'BG': 57, 'BIIB': 58, 'BK': 59, 'BKNG': 60, 'BKR': 61, 'BLDR': 62, 'BLK': 63, 'BMY': 64, 'BR': 65, 'BRO': 66, 'BSX': 67, 'BWA': 68, 'BX': 69, 'BXP': 70, 'C': 71, 'CAG': 72, 'CAH': 73, 'CARR': 74, 'CAT': 75, 'CB': 76, 'CBOE': 77, 'CBRE': 78, 'CCI': 79, 'CCL': 80, 'CDNS': 81, 'CDW': 82, 'CE': 83, 'CEG': 84, 'CF': 85, 'CFG': 86, 'CHD': 87, 'CHR

In [30]:
# Filtrar e remover as linhas com data maior que '2024-02-01'
df_filtered = df[df['Date'] < '2024-02-01']
df_filtered.tail()

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,SMA_20,SMA_50,...,Momentum_10,ATR,Volatility,Daily_Return,Future_Return,Rolling_Volatility,Year,Month,Day,Weekday
1648960,2024-01-31 00:00:00+00:00,58.132660,59.869999,60.669998,59.340000,60.209999,13842700,489,59.082282,59.161407,...,-0.213619,108.824397,1.835247,0.001841,-0.019456,0.008599,2024,1,31,4
1648961,2024-01-31 00:00:00+00:00,99.418686,102.809998,104.879997,102.769997,104.739998,22415300,490,96.930561,97.824467,...,5.637680,104.266225,2.523408,-0.019456,-0.011516,0.012724,2024,1,31,4
1648962,2024-01-31 00:00:00+00:00,111.212662,112.440002,114.300003,112.180000,113.919998,1304800,491,110.991111,108.332453,...,2.719978,97.639352,0.975211,-0.011516,-0.008651,0.011033,2024,1,31,4
1648963,2024-01-31 00:00:00+00:00,126.963341,129.490005,131.979996,129.259995,131.449997,2154200,492,127.178561,126.188563,...,0.401993,92.060827,1.095436,-0.008651,-0.018910,0.010799,2024,1,31,4
1648964,2024-01-31 00:00:00+00:00,186.389938,187.809998,192.779999,187.440002,192.130005,2150000,493,190.691774,187.653420,...,-2.637909,90.005767,3.556624,-0.018910,0.022675,0.013763,2024,1,31,4


In [31]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1648965 entries, 0 to 1648964
Data columns (total 25 columns):
 #   Column              Non-Null Count    Dtype              
---  ------              --------------    -----              
 0   Date                1648965 non-null  datetime64[ns, UTC]
 1   Adj Close           1648965 non-null  float64            
 2   Close               1648965 non-null  float64            
 3   High                1648965 non-null  float64            
 4   Low                 1648965 non-null  float64            
 5   Open                1648965 non-null  float64            
 6   Volume              1648965 non-null  int64              
 7   Ticker              1648965 non-null  int32              
 8   SMA_20              1648965 non-null  float64            
 9   SMA_50              1648965 non-null  float64            
 10  EMA_20              1648965 non-null  float64            
 11  RSI                 1648965 non-null  float64            
 12  BB_Mi

In [32]:
df_filtered.to_csv("Data_noNorm.csv", index=False)